# 1 Introducción

El siguiente ejemplo modifica el color de una imágen parametrizada, cambiandole el contraste. El cálculo de la se realiza aplicando la siguiente formula a cada color RGB del pixel:

<center>$ factorcontraste=(259.0 * (constraste + 255.0)) / (255.0 * (259.0 - contraste))$</center>

<center><br>$ valorescalado= (factorcontraste * (colorpixel  - 128) + 128)$</center>

EL objetivo es enseñar el funcionamiento del Lenguaje Python, CUDA y el manejo de imagenes a bajo nivel. El ejemplo es ilustrativo, ya que internamente el módulo Pillow posee varios filtros integrados.

-----
#Enunciado del Ejercicio

a)  Teniendo en cuenta el código secuencial del Filtro de Contraste de una imagen, que se ejecuta en la CPU, genere el kernel para ejecutar el mismo algoritmo en forma paralela en la gpu

---
## 2 Armado del ambiente
Toma la direcciòn web de una imagen con  acceso público en internet, la deja disponible al contexto de ejecuciòn del cuaderno colab.

In [ ]:
#@title # 2.1 Parámetros de ejecución
#@markdown ---
#@markdown ### Especifique la URL de la imagen:
url_imagen = "https://raw.githubusercontent.com/soa-pc-unlam/ProgramacionConcurrente/main/Enunciados%20TPs/TP-GPU/Imagenes/MK%201024x768.jpg" #@param {type:"string"}

#@markdown ---
# Leo la imagen desde internet.
#!wget https://github.com/wvaliente/SOA_HPC/blob/main/unlam.jpg?raw=true -O imagen.jpg

# TODO: Mejorar informaciòn y resutlado de ejecución.
!wget {url_imagen} -O imagen.jpg



---
## 2.2 Instala en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda

---
# 3 Desarrollo
# Ejecución en CPU
Ejecución del algoritmo del filtro de contraste ejecutandose en form secuencial en  la CPU

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import cProfile


max_value = 100 #Este seria el numero en porcentaje que se puede ingresar
new_range_max = 128 #Este seria el numero maximo de cantidad que se puede usar como contraste
CM= 1/2.54
SIZE= 12*CM

#Poner SHOW_STATS en true para ver el tiempo de ejecucion y false para ocultarlo
SHOW_STATS=True


def bounded_pixel_value(color, contrast_factor):
    scaled_value = (contrast_factor * (color  - 128) + 128)

    if scaled_value < 0:
        return 0
    elif scaled_value > 255:
        return 255

    return int(scaled_value)

def rangeValue(my_value):
    return (my_value/max_value)*new_range_max

#funcion que aplica el filtro de contraste
def filter_contrast(im,out,level_contrast):

  contrast= rangeValue (level_contrast)
  contrast_factor = (259.0 * (contrast + 255.0)) / (255.0 * (259.0 - contrast))

  width, height = im.size
  for x in range(width):
      for y in range(height):
          r,g,b = im.getpixel((x,y))

          updatedR = bounded_pixel_value(r, contrast_factor)
          updatedG = bounded_pixel_value(g, contrast_factor)
          updatedB = bounded_pixel_value(b, contrast_factor)

          out.putpixel((x,y), (updatedR, updatedG, updatedB))
  return out

def main():
  #objeto del Profiler de python
  cprofiler = cProfile.Profile()

  #se carga la imagen
  im = Image.open("imagen.jpg")
  out = Image.new('RGB', im.size, 0xffffff)

  #el usuario ingresa el procentaje de contraste aplicar
  level_contrast = float(input("Ingrese el porcentaje de contraste (-100 a 100): "))


  #se activa el profiler
  cprofiler.enable()

  #se aplica el filtro
  out= filter_contrast(im, out,level_contrast)

  #se desactiva el profiler
  cprofiler.disable()

  out.save('contrast_scaled.jpg')

  #se muestra la imagen original
  print("imagen original")
  plt.figure(figsize=(SIZE,SIZE))
  imgplot=plt.imshow( im)
  plt.show()

  #se muestra la imagen con filtro
  print("imagen con filtro")
  plt.figure(figsize=(SIZE,SIZE))
  img2plot=plt.imshow( out)
  plt.show()

  #se muestran los tiempos de ejecucion
  cprofiler.print_stats(sort='cumulative') if SHOW_STATS==True else None

main()

# Ejecución en GPU
Ejecución del algoritmo del filtro de contraste ejecutandose en forma paralela en  la GPU

In [ ]:
#descomentar la linea %%writefile filter_brightness.py y comentar la linea %matplotlib inline
#para medir el tiempo de ejecucion

#%%writefile filter_brightness.py
%matplotlib inline
from datetime import datetime
tiempo_total = datetime.now()

import matplotlib.pyplot as plt
import numpy
from PIL import Image
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule


#BRIGHTNESS_FACTOR  debe ser entre -100 y 100
CONTRAST_LEVEL = 100
CM= 1/2.54
SIZE= 12*CM

def define_kernel():

  module = SourceModule("""

    __global__ void kernel_img( int ancho, int alto, int *img_O, int *img_R, int level_contrast )
    {
      // Calculo las coordenadas del Thread en dos dimensiones.
      int idx = threadIdx.x + blockIdx.x*blockDim.x;
      int idy = threadIdx.y + blockIdx.y*blockDim.y;



      // Verifico que los Thread, esten dentro de las dimensiones de la imagen.
      if( (idx < ancho) && (idy< alto) )
      {


           **************************************************************
           ***********COMPLETE AQUI EL CODIGO DEL KERNEL*****************
           **************************************************************
      }
    }
    """)
  return module

def main():
  img_nombre = 'imagen.jpg'
  image = Image.open( img_nombre )

  # summarize some details about the image
  img_ancho, img_alto  = image.size

  # Convierto la imagen comprimida en JPEG/PNG a array
  img_O_cpu = numpy.asarray(image)
  img_O_cpu = img_O_cpu.astype( numpy.int32() )
  img_R_cpu = numpy.empty_like( img_O_cpu)

  # Reservo los 2 vectores en GPU(3 pixeles usa RGB * (el tamaño del array))
  img_O_gpu = cuda.mem_alloc( img_O_cpu.nbytes )
  img_R_gpu = cuda.mem_alloc( img_R_cpu.nbytes )

  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod( img_O_gpu, img_O_cpu )
  cuda.memcpy_htod( img_R_gpu, img_R_cpu )

  # CPU - Defino la función kernel que ejecutará en GPU.
  module = define_kernel()


  # Muestro los atributos de la imagen y como se ve antes del seudo filtro.
  print("Imagen del filtro: " + img_nombre + " -" + image.mode + "- [" + str(img_ancho) + ", " + str(img_alto ) + "]" )

  # CPU - Genero la función kernel.
  kernel = module.get_function("kernel_img")

  dim_hilo_x = 16
  dim_bloque_x = int( (img_ancho+dim_hilo_x-1) / dim_hilo_x )

  dim_hilo_y = 19
  dim_bloque_y = int( (img_alto+dim_hilo_y-1) / dim_hilo_y )

  print( "Thread: [", dim_hilo_x, ",", dim_hilo_y, " ], Bloque : [", dim_bloque_x, ",", dim_bloque_y, "]" )
  print( "Total de Thread: [", dim_hilo_x*dim_bloque_x, ",", dim_hilo_y*dim_bloque_y, " ]", " = ", dim_hilo_x*dim_bloque_x*dim_hilo_y*dim_bloque_y )


  kernel( numpy.int32(img_ancho), numpy.int32(img_alto), img_O_gpu, img_R_gpu,numpy.int32(CONTRAST_LEVEL), block=( dim_hilo_x, dim_hilo_y, 1 ), grid=(dim_bloque_x, dim_bloque_y,1))

  # GPU - Copio el resultado desde la memoria GPU.
  cuda.memcpy_dtoh( img_R_cpu, img_R_gpu )

  # Muestro la imagen Original el filtro.
  plt.figure(figsize=(SIZE,SIZE))
  imgplot=plt.imshow( img_O_cpu )

  # Muestro la imagen luego de aplicarle el filtro.
  plt.figure(figsize=(SIZE,SIZE))
  imgplot=plt.imshow( img_R_cpu )

main()

In [ ]:
!nvprof python filter_brightness.py

---
# 4 Tabla de pasos


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  wget url_imagen       | Lectura de la direcciòn URL de la imagen (jpg) a procesar.
CPU      | pip install pycuda    | Instala en el cuaderno los driver de CUDA para Python.
CPU      |  matplotlib inline    | Macro de Colab para mostrar imagenes.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  Image.open()          | Abre el archivo de la imagen.
CPU      |  numpy.asarray(imagen) | Convierte el formato comprimido JPG a RAW.
CPU      |  numpy.empty_like(()   | Genera el array destino, que tendrá a la imagen resultado.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria para las imagenes en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copio los valores en crudo de las imagenes al GPU.
CPU      |  SourceModule()        | Posee el còdigo del kernel.
CPU      |  module.get_function() | convierte el texto del kernel en funcion de Python.
CPU      |  dim_hilo_x, dim_hilo_y| Calcula las dimensiones para la ejecuciòn de 2D.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU, enviando los parametros.
CPU      |  print()               | Informa los atributos de la imagen.
CPU      | cuda.memcpy_dtoh()     | Copia desde la memoria GPU al CPU.
CPU      |  plt.imshow            | Muestra la imagen original.
CPU      |  plt.imshow            | Muestra la imagen resultado.



---
# 5 Conclusiones

Las conclusiones son explicadas en clase...

---
# 6 Bibliografía

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb)

[3] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[4] Algoritmo de filtro de Contraste en forma secuencial [PDF](https://www.dfstudios.co.uk/articles/programming/image-programming-algorithms/image-processing-algorithms-part-5-contrast-adjustment/)
